In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mlp import Layer, MultilayerPerceptron, SquaredError, Relu, Linear
from sklearn.metrics import r2_score

# -------------------- Loading MPG Dataset -------------------- #
class MPGDataLoader:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path

    def load_data(self):
        columns = ["mpg", "cylinders", "displacement", "horsepower", "weight",
                   "acceleration", "model year", "origin", "car name"]
        df = pd.read_csv(self.dataset_path, sep='\s+', names=columns, na_values="?")
        df.dropna(inplace=True)
        df.drop(columns=["car name"], inplace=True, errors='ignore')
        df["horsepower"] = pd.to_numeric(df["horsepower"], errors='coerce')
        df.dropna(inplace=True)

        y = df["mpg"].values.reshape(-1, 1)
        X = df.drop(columns=["mpg"]).values
        return X, y

# Load dataset
dataset_path = 'mpg-data/auto-mpg.data'
dataloader = MPGDataLoader(dataset_path)
X, y = dataloader.load_data()

# -------------------- Data Preprocessing -------------------- #
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_x.fit_transform(X_train)
X_val = scaler_x.transform(X_val)
X_test = scaler_x.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)

input_dim = X_train.shape[1]

# -------------------- Hyperparameter Tuning: Dropout -------------------- #
dropout_rates = [0.0, 0.2, 0.4, 0.5, 0.6]

results = {}

for dropout in dropout_rates:
    print(f"\nTraining model with dropout = {dropout}...")

    layers = [
        Layer(input_dim, 64, Relu(), dropout_rate=dropout),
        Layer(64, 64, Relu(), dropout_rate=dropout),
        Layer(64, 64, Relu(), dropout_rate=dropout),
        Layer(64, 1, Linear())
    ]

    mlp = MultilayerPerceptron(layers)
    train_losses, val_losses = mlp.train(
        X_train, y_train, X_val, y_val,
        SquaredError(), learning_rate=0.001, batch_size=32, epochs=150, optimizer='vanilla', momentum=0.7
    )

    y_pred = mlp.forward(X_test, training=False)
    y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))
    y_test_original = scaler_y.inverse_transform(y_test)

    r2 = r2_score(y_test_original, y_pred)

    results[dropout] = round(r2 * 100, 2)
    print(f"R² Score for dropout {dropout}: {results[dropout]}%")

# Identify Best Dropout Rate
best_dropout = max(results, key=results.get)
best_r2 = results[best_dropout]

# -------------------- Summary of Results -------------------- #
print("\n===== Summary of Runs =====")
for dr, r2 in results.items():
    print(f"R² value for dropout {dr}: {r2}%")

print(f"\nBest Dropout: {best_dropout} with R² = {best_r2}%")




Training model with dropout = 0.0...
Epoch 1/150 - Train Loss: 1.1756, Val Loss: 0.9352
Epoch 2/150 - Train Loss: 0.9559, Val Loss: 0.7498
Epoch 3/150 - Train Loss: 0.7607, Val Loss: 0.6143
Epoch 4/150 - Train Loss: 0.6393, Val Loss: 0.5180
Epoch 5/150 - Train Loss: 0.5388, Val Loss: 0.4429
Epoch 6/150 - Train Loss: 0.4702, Val Loss: 0.3847
Epoch 7/150 - Train Loss: 0.4031, Val Loss: 0.3384
Epoch 8/150 - Train Loss: 0.3610, Val Loss: 0.3025
Epoch 9/150 - Train Loss: 0.3171, Val Loss: 0.2748
Epoch 10/150 - Train Loss: 0.2993, Val Loss: 0.2541
Epoch 11/150 - Train Loss: 0.2725, Val Loss: 0.2379
Epoch 12/150 - Train Loss: 0.2524, Val Loss: 0.2249
Epoch 13/150 - Train Loss: 0.2446, Val Loss: 0.2144
Epoch 14/150 - Train Loss: 0.2368, Val Loss: 0.2052
Epoch 15/150 - Train Loss: 0.2256, Val Loss: 0.1972
Epoch 16/150 - Train Loss: 0.2177, Val Loss: 0.1902
Epoch 17/150 - Train Loss: 0.2166, Val Loss: 0.1839
Epoch 18/150 - Train Loss: 0.2086, Val Loss: 0.1780
Epoch 19/150 - Train Loss: 0.2085, 